In [ ]:
"""
https://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/

see more about this tab_delimited file at:https://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/README
"""
import pandas as pd 
import re 
from typing import List
from functools import partial
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)


: 

In [2]:




def strip_comment_head(df:pd.DataFrame, comment="#", inplace=True)  -> pd.DataFrame | None:
    """去除header列可能存在的注释符号，如：#accession .... ，这里#可以是任意的comment符号

    Args:
        df (_type_): _description_
        comment (str, optional): _description_. Defaults to "#".
        inplace (bool, optional): _description_. Defaults to True.

    Returns:
        _type_: _description_
    """
    if comment in df.columns[0]:
        if inplace:
            df.rename(columns={df.columns[0]: re.split(f"{comment}" + r"[\s]*", df.columns[0])[-1]}, inplace=True)
        else:
            return df.rename(columns={df.columns[0]: re.split(f"{comment}" + r"[\s]*", df.columns[0])[-1]}, inplace=False) 
    else:
        return df 


def splitName2Attribution(x):
    """
    splitName2Attribution HGSV 命名法：https://varnomen.hgvs.org/recommendations/general/
    refSeq:https://en.wikipedia.org/wiki/RefSeq#:~:text=The%20Reference%20Sequence%20(RefSeq)%20database,was%20first%20introduced%20in%202000.

    Args:
        str类型: "NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter)"

    Returns:
        ["NM_017547.4", "Gln232Ter","694C>T"]
    """
    ref_pattern = r"[A-Za-z]+_[0-9]+[\.]*[0-9]*"
    AAS_pattern = r"(?<=(p.))[A-Za-z]{0,3}[^\s()]*"
    SNP_pattern = r"(?<=(c.))[\d]*[^\s()]*"
    ref = re.search(ref_pattern, x).group() if re.search(ref_pattern, x) else None 
    AAS = re.search(AAS_pattern, x).group() if  re.search(AAS_pattern, x) else None 
    SNP = re.search(SNP_pattern, x).group() if re.search(SNP_pattern, x) else None 
    return ref, AAS, SNP 

def keep_nsSNP(clinvar_variant_summary_snp:pd.DataFrame, subset:str = "Name"):
    """keep nsSNP from clinvar variant summary file, subset is the column of HGVS, it should be Name columns, check https://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/README for more.

    Returns:
        pd.DataFrame: nsSNP
    """
    clinvar_variant_summary_snp = clinvar_variant_summary_snp.dropna(axis=0, subset=[subset])  # remove NA at Name column

    #  get HGVS of SAV at Name, remove NA entry at AAS 
    tmp = clinvar_variant_summary_snp[subset].apply(lambda x: splitName2Attribution(x))
    clinvar_variant_summary_snp["ref_seq"] = tmp.apply(lambda x: x[0])
    clinvar_variant_summary_snp["AAS"] = tmp.apply(lambda x: x[1])
    clinvar_variant_summary_snp["SNP"] = tmp.apply(lambda x: x[2])

    clinvar_variant_summary_snp = clinvar_variant_summary_snp[~clinvar_variant_summary_snp["AAS"].isna()]  # drop na

    #  remove synonymous SNP at protein coding region from data
    clinvar_variant_summary_sav:pd.DataFrame = clinvar_variant_summary_snp[~clinvar_variant_summary_snp["AAS"].apply(lambda x : "=" in x)]

    del clinvar_variant_summary_sav["ref_seq"]
    del clinvar_variant_summary_sav["AAS"]
    del clinvar_variant_summary_sav["SNP"]

    return clinvar_variant_summary_sav.reset_index(drop=True)

def keep_clinicalsignificance(clinvar_variant_summary:pd.DataFrame, fields:List):
    """keep SNV, if SNV's ClinicalSignificance in fields

    Args:
        clinvar_variant_summary (pd.DataFrame): _description_
        fields (List): list of ClinicalSignificance
    Return:
        filtered clinvar_variant_summary 
    """
    def filter_func(x:str, fields:List):
        x = str.lower(x)
        if x in fields:
            return True
        else:
            return False
    if isinstance(fields, str):
        fields = [fields]
        
    fields = [str.lower(i) for i in fields]
    func = partial(filter_func, fields = fields)

    return clinvar_variant_summary[clinvar_variant_summary["ClinicalSignificance"].apply(func)].reset_index(drop=True)

In [4]:
clinvar_variant_summary_dir = "/p300s/wangmx_group/xutingfeng/statistic/proture/data/clinvar/variant_summary_2018-02.txt.gz"


assembly = "GRCh37"




In [5]:
clinvar_variant_summary = pd.read_csv(clinvar_variant_summary_dir, sep="\t", compression="gzip")
clinvar_variant_summary = strip_comment_head(clinvar_variant_summary, inplace=False)
print(f"loading clinvar data with shape:{clinvar_variant_summary.shape}")
# keep Assembly 
clinvar_variant_summary = clinvar_variant_summary[clinvar_variant_summary["Assembly"] == assembly]
print(f"keep assembly from {assembly}, output is {clinvar_variant_summary.shape}")

# keep SNP
clinvar_variant_summary_snp = clinvar_variant_summary[clinvar_variant_summary["Type"] == "single nucleotide variant"]
print(f"keep SNP and the output is {clinvar_variant_summary_snp.shape}")

# keep nsSNP
clinvar_variant_summary_snp = keep_nsSNP(clinvar_variant_summary_snp)
print(f"keep nsSNP and the output is {clinvar_variant_summary_snp.shape}")

# keep Pathogenic 
fields=["Pathogenic", "Likely Pathogenic"]
clinvar_pathogenic_variant_summary_snp = keep_clinicalsignificance(clinvar_variant_summary = clinvar_variant_summary_snp, fields=fields)
print(f"keep {', '.join(fields)} and the output is {clinvar_pathogenic_variant_summary_snp.shape}")


loading clinvar data with shape:(748078, 30)
keep assembly from GRCh37, output is (365665, 30)
keep SNP and the output is (301643, 30)
keep nsSNP and the output is (159146, 30)
keep Pathogenic, Likely Pathogenic and the output is (37304, 30)


In [71]:
def ReviewStatus2star(x):
    if x =="practice guideline":
        return 4
    if x =="reviewed by expert panel":
        return 3
    if x =="criteria provided, multiple submitters, no conflicts":
        return 2
    if x == "criteria provided, conflicting interpretations" or x == "criteria provided, single submitter":
        return 1
    if x == "no assertion criteria provided" or x == "no assertion provided" or x == "no interpretation for the single variant":
        return 0
clinvar_pathogenic_variant_summary_snp["star"] = clinvar_pathogenic_variant_summary_snp["ReviewStatus"].apply(ReviewStatus2star)
clinvar_pathogenic_variant_summary_snp["star"].value_counts()

0.0    17360
1.0    16142
2.0     2298
3.0     1465
4.0       12
Name: star, dtype: int64

In [14]:
import os.path as osp

In [17]:
osp.splitext("dadada.csv")

('dadada', '.csv')

In [6]:
clinvar_pathogenic_variant_summary_snp

,AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),nsv/esv (dbVar),RCVaccession,PhenotypeIDS,PhenotypeList,Origin,OriginSimple,Assembly,ChromosomeAccession,Chromosome,Start,Stop,ReferenceAllele,AlternateAllele,Cytogenetic,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories
0,15044,single nucleotide variant,NM_017547.3(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Oct 01, 2010",267606829,-,RCV000000015,"MedGen:C1838979,OMIM:252010",Mitochondrial complex I deficiency,germline,germline,GRCh37,NC_000011.9,11,126145284,126145284,C,T,11q24,no assertion criteria provided,1,NaN,N,OMIM Allelic Variant:613622.0001,1
1,15045,single nucleotide variant,NM_017547.3(FOXRED1):c.1289A>G (p.Asn430Ser),55572,FOXRED1,HGNC:26927,Pathogenic,1,"Oct 01, 2010",267606830,-,RCV000000016,"MedGen:C1838979,OMIM:252010",Mitochondrial complex I deficiency,germline,germline,GRCh37,NC_000011.9,11,126147412,126147412,A,G,11q24,no assertion criteria provided,1,NaN,N,"OMIM Allelic Variant:613622.0002,UniProtKB (pr...",1
2,15051,single nucleotide variant,NM_000410.3(HFE):c.314T>C (p.Ile105Thr),3077,HFE,HGNC:4886,Pathogenic,1,"Jun 01, 1999",28934596,-,RCV000000029,"MedGen:C3469186,OMIM:235200",Hemochromatosis type 1,germline,germline,GRCh37,NC_000006.11,6,26091306,26091306,T,C,6p22.2,no assertion criteria provided,1,NaN,N,"OMIM Allelic Variant:613609.0009,UniProtKB (pr...",1
3,15052,single nucleotide variant,NM_000410.3(HFE):c.277G>C (p.Gly93Arg),3077,HFE,HGNC:4886,Pathogenic,1,"Jun 01, 1999",28934597,-,RCV000000030,"MedGen:C3469186,OMIM:235200",Hemochromatosis type 1,germline,germline,GRCh37,NC_000006.11,6,26091269,26091269,G,C,6p22.2,no assertion criteria provided,1,NaN,N,"OMIM Allelic Variant:613609.0010,UniProtKB (pr...",1
4,15056,single nucleotide variant,NM_000410.3(HFE):c.381A>C (p.Gln127His),3077,HFE,HGNC:4886,Pathogenic,1,"Aug 01, 1999",28934595,-,RCV000000034,"MedGen:C3469186,OMIM:235200",Hemochromatosis type 1,germline,germline,GRCh37,NC_000006.11,6,26091582,26091582,A,C,6p22.2,no assertion criteria provided,1,NaN,N,"OMIM Allelic Variant:613609.0007,UniProtKB (pr...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37299,481081,single nucleotide variant,NM_001110556.1(FLNA):c.853C>T (p.Arg285Cys),2316,FLNA,HGNC:3754,Pathogenic,1,-,-1,-,RCV000577903,"MedGen:C1848213,OMIM:300049,OMIM:300537,SNOMED...",Periventricular nodular heterotopia 1,inherited,germline,GRCh37,NC_000023.10,X,153595780,153595780,G,A,Xq28,no assertion criteria provided,1,NaN,N,-,2
37300,481082,single nucleotide variant,NM_001110556.1(FLNA):c.82A>G (p.Met28Val),2316,FLNA,HGNC:3754,Pathogenic,1,-,-1,-,RCV000577884,"MedGen:C1848213,OMIM:300049,OMIM:300537,SNOMED...",Periventricular nodular heterotopia 1,de novo,germline,GRCh37,NC_000023.10,X,153599532,153599532,T,C,Xq28,no assertion criteria provided,1,NaN,N,-,2
37301,481084,single nucleotide variant,NM_000049.2(ASPA):c.604G>C (p.Ala202Pro),443,ASPA,HGNC:756,Likely pathogenic,1,"Sep 28, 2017",147763700,-,RCV000577927,"MedGen:C0206307,OMIM:271900,Orphanet:ORPHA141,...",Spongy degeneration of central nervous system,germline,germline,GRCh37,NC_000017.10,17,3392606,3392606,G,C,17p13.2,"criteria provided, single submitter",1,NaN,N,-,2
37302,481124,single nucleotide variant,NM_015120.4(ALMS1):c.5146A>T (p.Arg1716Ter),7840,ALMS1,HGNC:428,Likely pathogenic,1,"Aug 01, 2017",773513360,-,RCV000578023,"MedGen:C0268425,OMIM:203800,Orphanet:ORPHA64,S...",Alstrom syndrome,germline,germline,GRCh37,NC_000002.11,2,73678797,73678797,A,T,2p13.1,"criteria provided, single submitter",1,NaN,N,-,2


In [12]:
clinvar_variant_summary = clinvar_variant_summary[~clinvar_variant_summary["Name"].isna()]

In [14]:
clinvar_variant_summary[clinvar_variant_summary["Name"].apply(lambda x: "=" in x if x is not None else False)]

,AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,ClinSigSimple,LastEvaluated,RS# (dbSNP),nsv/esv (dbVar),RCVaccession,PhenotypeIDS,PhenotypeList,Origin,OriginSimple,Assembly,ChromosomeAccession,Chromosome,Start,Stop,ReferenceAllele,AlternateAllele,Cytogenetic,ReviewStatus,NumberSubmitters,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories
139,15111,single nucleotide variant,NM_000374.4(UROD):c.942G>A (p.Glu314=),7389,UROD,HGNC:12591,Pathogenic,1,"Nov 01, 1998",121918062,-,RCV000000090,"MedGen:C0268323,OMIM:176100,SNOMED CT:59229005",Familial porphyria cutanea tarda,germline,germline,GRCh37,NC_000001.10,1,45480678,45480678,G,A,1p34.1,no assertion criteria provided,1,NaN,N,OMIM Allelic Variant:613521.0008,1
311,15200,single nucleotide variant,NM_000274.3(OAT):c.1134C>T (p.Asn378=),4942,OAT,HGNC:8091,Benign,0,"Jun 14, 2016",11461,-,RCV000000184;RCV000380778,"na;MedGen:C0599035,OMIM:258870,Orphanet:ORPHA4...",OAT POLYMORPHISM;Ornithine aminotransferase de...,germline,germline,GRCh37,NC_000010.10,10,126089434,126089434,G,A,10q26.1,"criteria provided, single submitter",2,NaN,N,OMIM Allelic Variant:613349.0017,3
1121,15640,single nucleotide variant,NM_000277.2(PAH):c.1197A>T (p.Val399=),5053,PAH,HGNC:8582,Pathogenic,1,"Jan 01, 2001",199475584,-,RCV000000632;RCV000088786,"MedGen:C0031485,OMIM:261600,Orphanet:ORPHA716,...",Phenylketonuria;not provided,germline,germline,GRCh37,NC_000012.11,12,103237426,103237426,T,A,12q23.2,no assertion criteria provided,2,NaN,N,"OMIM Allelic Variant:612349.0027,OMIM Allelic ...",3
1347,15763,single nucleotide variant,NM_000015.2(NAT2):c.803G= (p.Arg268=),10,NAT2,HGNC:7646,drug response,0,"Oct 28, 2012",1208,-,RCV000000760,"MedGen:C1827377,OMIM:243400,SNOMED CT:425079005",Slow acetylator due to N-acetyltransferase enz...,germline,germline,GRCh37,NC_000008.10,8,18258316,18258316,G,G,8p22,no assertion criteria provided,1,NaN,N,OMIM Allelic Variant:612182.0003,1
2436,16365,single nucleotide variant,NM_024296.4(CCDC28B):c.330C>T (p.Phe110=),79140,CCDC28B,HGNC:28163,risk factor,0,"Oct 15, 2008",41263993,-,RCV000001389,na,"Bardet-Biedl syndrome 1, modifier of",germline;inherited,germline,GRCh37,NC_000001.10,1,32669645,32669645,C,T,1p35.1,no assertion criteria provided,2,NaN,N,OMIM Allelic Variant:610162.0001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747588,480428,single nucleotide variant,NM_000296.3(PKD1):c.2109C>T (p.His703=),5310,PKD1,HGNC:9008,Benign,0,"Jun 09, 2017",527655141,-,RCV000576311,"MedGen:C0085413,OMIM:173900,SNOMED CT:28728008","Polycystic kidney disease, adult type",germline,germline,GRCh37,NC_000016.9,16,2164915,2164915,G,A,16p13.3,"criteria provided, single submitter",1,NaN,N,-,2
747594,480431,single nucleotide variant,NM_006946.3(SPTBN2):c.1416G>A (p.Thr472=),6712,SPTBN2,HGNC:11276,Benign,0,"Jun 29, 2017",145249947,-,RCV000576735,"MedGen:C0752123,OMIM:600224,Orphanet:ORPHA98766",Spinocerebellar ataxia 5,germline,germline,GRCh37,NC_000011.9,11,66475224,66475224,C,T,11q13.2,"criteria provided, single submitter",1,NaN,N,-,2
747595,480432,single nucleotide variant,NM_002087.3(GRN):c.42G>A (p.Leu14=),2896,GRN,HGNC:4601,Benign,0,"Jun 23, 2017",111435385,-,RCV000576580,"MedGen:C1843792,OMIM:607485","Frontotemporal dementia, ubiquitin-positive",germline,germline,GRCh37,NC_000017.10,17,42426574,42426574,G,A,17q21.31,"criteria provided, single submitter",1,NaN,N,-,2
747780,480542,single nucleotide variant,NM_002215.3(ITIH1):c.1754A= (p.Glu585=),3697,ITIH1,HGNC:6166,-,0,"Apr 01, 1995",678,-,RCV000015920;RCV000015922,na;na,"INTER-ALPHA-TRYPSIN INHIBITOR, HEAVY CHAIN 1 P...",germline,germline,GRCh37,NC_000003.11,3,52820981,52820981,A,A,3p21.1,no assertion for the individual variant,1,NaN,N,"OMIM Allelic Variant:147270.0001,OMIM Allelic ...",1
